In [1]:
%run utils
import utils

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from tqdm.notebook import trange, tqdm

In [4]:
TEST_FILE = "data/splits/val_inference_both.csv"
training_both = "data/splits/training_set_naip_nlcd_both.csv"
val_both = "data/splits/val_inference_both.csv"
IMAGE_COL = "image_fn"
LABEL_COL = "label_fn"
GROUP_COL = "gropu"
ROW = 3

### What are the dimensions of NAIP/NLCD image?

In [5]:
df = pd.read_csv(TEST_FILE)
a = io.imread(df.loc[3, LABEL_COL])
a.shape

(3880, 3880)

In [6]:
df = pd.read_csv(TEST_FILE)
a = io.imread(df.loc[3, IMAGE_COL])
a.shape

(3880, 3880, 4)

### Are validation images also in the training set?

In [7]:
training_both_df = pd.read_csv(training_both)
val_both_df = pd.read_csv(val_both)
print(f"There are {len(training_both_df)} samples in the training set")
print(f"There are {len(val_both_df)} samples in the val set")

There are 4500 samples in the training set
There are 100 samples in the val set


In [8]:
all_in = True
for i in range(len(val_both_df)):
    if val_both_df.loc[i, "image_fn"] not in training_both_df["image_fn"].tolist():
        print(val_both_df.loc[i, "image_fn"])
        all_in = False
        break
    elif val_both_df.loc[i, "label_fn"] not in training_both_df["label_fn"].tolist():
        print(val_both_df.loc[i, "image_fn"])
        all_in = False
        break
all_in

True

### Then we could only focus on the training data

In [9]:
def compute_percentage(df, df_row, img_ndarray) -> None:
    """Do one_pass to fill in df columns"""
    for i in range(img_ndarray.shape[0]):
        for j in range(img_ndarray.shape[1]):
            df.loc[df_row, utils.NLCD_CLASS[img_ndarray[i][j]]] += 1
    for class_name in utils.NLCD_CLASS.values():
        df.loc[df_row, class_name] /= (img_ndarray.shape[0] * img_ndarray.shape[1])

In [10]:
def transform_df(sample_df) -> pd.DataFrame:
    """Transform training_df or val_df such that it one-hot-encodes 
    the percentage of each class in each image, done for the NLCD column"""
    df_copy = sample_df.copy()
    for v in utils.NLCD_CLASS.values():
        df_copy[v] = 0
    for i in range(len(sample_df)):
        img = io.imread(df_copy.loc[i, "label_fn"])
        compute_percentage(df_copy, i, img)
    return df_copy

In [ ]:
def plot_histogram_one_class(sample_df, class_int):
    """Plot distribution of a certain class throughout samples
    
    Args:
        sample_df: either training_df or val_df
        class_int: the integer representing the class
    """
    

In [12]:
df_copy = training_both_df.copy()
for v in utils.NLCD_CLASS.values():
    df_copy[v] = 0
for i in trange(len(df_copy)):
    img = io.imread(df_copy.loc[i, "label_fn"])
    compute_percentage(df_copy, i, img)
df_copy.to_csv("training_both_encoded.csv")

  0%|          | 0/4500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# plot a distribution of the percentage of a certain class in a certain image
